In [1]:
import json
import os
from decimal import Decimal
from typing import Annotated, Dict

from dotenv import load_dotenv
from web3 import HTTPProvider, Web3

import autogen

In [2]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "tags": ["tool"],
    },
)

llm_config = {
    "config_list": config_list,
}

print("Model used: " + llm_config["config_list"][0]["model"])

Model used: gpt-4


In [7]:
# Load environment variables from .env file

import random

load_dotenv()

# Get environment variables
PROVIDER_URL = os.getenv("PROVIDER_URL", "http://localhost:8545")
PRIVATE_KEY_1 = os.getenv("PRIVATE_KEY_1", "some random key 1")
PRIVATE_KEY_2 = os.getenv("PRIVATE_KEY_2", "some random key 2")


# Let's first define the assistant agent that suggests tool calls.
assistant = autogen.ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. " "Return 'TERMINATE' when the task is done.",
    llm_config=llm_config,
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = autogen.ConversableAgent(
    name="User",
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

w3: Dict[str, Web3] = {}

# Connect to an Ethereum node


def connect_to_eth() -> str:
    global w3
    id = f"w3-{random.randint(0, 1000000):06d}"
    try:
        w3[id] = Web3(HTTPProvider(PROVIDER_URL))
    except Exception as e:
        return f"Failed to connect to the Ethereum node: {e}"

    return id


# Get the current block number


def get_block_number(
    connection_id: Annotated[str, "ID of the Etherium node connection. Call `connect_to_eth` to get it."]
) -> int:
    global w3
    if connection_id not in w3:
        return "Connection ID not found. Call `connect_to_eth` to get the ID of the connection."

    return w3[connection_id].eth.block_number


# Register the tool functions with the assistant agent.
autogen.register_function(
    connect_to_eth,
    caller=assistant,
    executor=user_proxy,
    name="connect_to_eth",
    description="Connect to an Ethereum node",
)
autogen.register_function(
    get_block_number,
    caller=assistant,
    executor=user_proxy,
    name="get_block_number",
    description="Get the current block number",
)

chat_result = user_proxy.initiate_chat(
    assistant, message="Connect to an Ethereum node and get the current block id.", summary_method="reflection_with_llm"
)

print(chat_result.summary)

User (to Assistant):

Connect to an Ethereum node and get the current block id.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_mPyH5WzYp8Li9z1B8DpSkCjU): connect_to_eth *****
Arguments: 
{}
*******************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION connect_to_eth...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_mPyH5WzYp8Li9z1B8DpSkCjU) *****
w3-864280
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_zGKLQWgH55d6wJzW9UWvWcDj): get_block_number *****
Arguments: 
{
  "connection_id": "w3-864280"
}
***********************